# AfterWork Data Science: Getting Started with NLP Project

### Prerequisites

In [ ]:
# Importing the required libraries
# ---
# 
import pandas as pd # library for data manipulation
import numpy as np  # librariy for scientific computations
import re           # regex library to perform text preprocessing
import string       # library to work with strings
import nltk         # library for natural language processing
import scipy        # scientific conputing 

### 1. Importing our Data

In [ ]:
# Question: Given a new tweets, create a sentiment analysis model that will 
# predict whether a tweet will contain positive or negative sentiment.
# ---
# Dataset url = https://bit.ly/31kqByD 
# ---
#
data = pd.read_csv('https://bit.ly/31kqByD', encoding='latin-1')

In [ ]:
df = data.copy()
df.head()

,Unnamed: 0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my d..."


### 2. Data Exploration

In [ ]:
# We can determine the size of our dataset
# ---
#
df.shape

(10000, 7)

Seems this dataset will need some data cleaning i.e. columns. We also don't need some columns to perform create our model. We will drop those columns.

### 3. Data Preparation

#### Basic Data Cleaning Techniques

In [ ]:
# We rename the columns for ease of referencing our columns later on
# ---
#
df.columns = ['id', 'target', 't_id', 'created_at', 'query', 'user', 'text']
df.head()

,id,target,t_id,created_at,query,user,text
0,346508,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
1,883537,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
2,764173,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
3,638701,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...
4,664821,0,2244623416,Fri Jun 19 14:59:46 PDT 2009,NO_QUERY,reinventingjess,"Crap, I was counting down the hours until my d..."


In [ ]:
# We retain the relevant columns by dropping the columns we don't need 
# for creating a sentiment analysis model. 
# ---
#
df = df.drop(['id', 't_id', 'created_at', 'query', 'user'], axis = 1)
df.head()

,target,text
0,0,Obama forges his Muslim alliance against the c...
1,4,Had the most spectacular prom ever but now my...
2,0,I am overwhelmed today taking a moment to eat...
3,0,@lindork Tres sad. I was totally a Max fan. #...
4,0,"Crap, I was counting down the hours until my d..."


In [ ]:
# Understanding the distribution of target
# ---
#
df.target.value_counts() 

0    5067
4    4933
Name: target, dtype: int64

In [ ]:
# Let's determine whether our columns have the right data types
# ---
#
df.dtypes

target     int64
text      object
dtype: object

In [ ]:
# What values are in our target variable?
# ---
#
df.target.unique()

array([0, 4], dtype=int64)

These are the two classes to which each document (text) belongs. The target value 0 means a text with a negative sentiment, while that of 4 means a text with a positive sentiment. 

In [ ]:
# Let's check for missing values 
# ---
# 
df.isnull().sum()

target    0
text      0
dtype: int64

We don't have any missing values, so we are good to go.

#### Text Processing

In [ ]:
# Text Cleaning: Removing all urls/links
# ---
# 
df['text'] =  df['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+','', str(x)))
df[['text']].head()

,text
0,Obama forges his Muslim alliance against the c...
1,Had the most spectacular prom ever but now my...
2,I am overwhelmed today taking a moment to eat...
3,@lindork Tres sad. I was totally a Max fan. #...
4,"Crap, I was counting down the hours until my d..."


In [ ]:
# Text Cleaning: Removing @ and # characters or replace them with space
# ---
df['text'] =  df['text'].apply(lambda x: re.sub(r'@|#','', str(x)))
df[['text']].head()

,text
0,Obama forges his Muslim alliance against the c...
1,Had the most spectacular prom ever but now my...
2,I am overwhelmed today taking a moment to eat...
3,lindork Tres sad. I was totally a Max fan. SY...
4,"Crap, I was counting down the hours until my d..."


In [ ]:
# Text Cleaning: Conversion to lowercase
# ---
df['text'] =  df['text'].apply(str.lower)
df[['text']].head()

,text
0,obama forges his muslim alliance against the c...
1,had the most spectacular prom ever but now my...
2,i am overwhelmed today taking a moment to eat...
3,lindork tres sad. i was totally a max fan. sy...
4,"crap, i was counting down the hours until my d..."


In [ ]:
# Installing wordnija and textblob
# ---
#
!pip install wordninja
!pip install textblob

# Importing those libraries
# ---
#
import wordninja
import textblob

In [ ]:
df.head()

,target,text
0,0,obama forges his muslim alliance against the c...
1,4,had the most spectacular prom ever but now my...
2,0,i am overwhelmed today taking a moment to eat...
3,0,lindork tres sad. i was totally a max fan. sy...
4,0,"crap, i was counting down the hours until my d..."


In [ ]:
# Text Cleaning: Removing punctuation characters
# ---
df['text'] =  df['text'].apply(lambda x: re.sub(r'\W+',' ', str(x)))
df[['text']].head()

,text
0,obama forges his muslim alliance against the c...
1,had the most spectacular prom ever but now my ...
2,i am overwhelmed today taking a moment to eat ...
3,lindork tres sad i was totally a max fan sytycd
4,crap i was counting down the hours until my da...


In [ ]:
# Text Cleaning: Removing stop words
# ---
def remove_stopword(txt):
    stopwords = nltk.corpus.stopwords.words("english")
    txt = txt.split(".")
    txt = [i for i in txt if i not in stopwords]
    return txt

df['text'] =  df['text'].apply(remove_stopword)
df[['text']].head()

,text
0,[obama forges his muslim alliance against the ...
1,[had the most spectacular prom ever but now my...
2,[i am overwhelmed today taking a moment to eat...
3,[lindork tres sad i was totally a max fan sytycd]
4,[crap i was counting down the hours until my d...


In [ ]:
# Text Cleaning: Lemmatization
# ---
def lemmatize(txt):
    txt = txt[0].split(" ")
    lemma = nltk.stem.WordNetLemmatizer()
    txt = [lemma.lemmatize(i) for i in txt]
    txt = " ".join(txt)
    return txt
# For lemmatization, we will need to download wordnet
#
nltk.download("wordnet")

# Lemmatizing our text
# ---
#
df['text'] =  df['text'].apply(lemmatize)
df[['text']].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text
0,obama forge his muslim alliance against the ci...
1,had the most spectacular prom ever but now my ...
2,i am overwhelmed today taking a moment to eat ...
3,lindork tres sad i wa totally a max fan sytycd
4,crap i wa counting down the hour until my dad ...


We won't remove numerics because we could loose meaning of our text if we lost the numerics. We could also further prepare our text by performing spelling correction but this is a resource intensive process that we will skip for now.

#### Feature Engineering Techniques 

In [ ]:
# Feature Construction: Length of tweet
# ---
# YOUR CODE GOES BELOW
#

df['text_len'] =  df['text'].apply(len)
df.head()

,target,text,text_len
0,0,obama forge his muslim alliance against the ci...,104
1,4,had the most spectacular prom ever but now my ...,127
2,0,i am overwhelmed today taking a moment to eat ...,55
3,0,lindork tres sad i wa totally a max fan sytycd,46
4,0,crap i wa counting down the hour until my dad ...,132


In [ ]:
# Feature Construction: Word count 
# ---
# YOUR CODE GOES BELOW
# 
def find_count(txt):
    txt = txt.split(" ")
    return len(txt)
df['words'] =  df['text'].apply(find_count)
df.head()

,target,text,text_len,words
0,0,obama forge his muslim alliance against the ci...,104,22
1,4,had the most spectacular prom ever but now my ...,127,25
2,0,i am overwhelmed today taking a moment to eat ...,55,12
3,0,lindork tres sad i wa totally a max fan sytycd,46,10
4,0,crap i wa counting down the hour until my dad ...,132,30


In [ ]:
# Feature Construction: Word density (Average no. of words / tweet)
# ---
# YOUR CODE GOES BELOW
#
df["density"] = df.words/df.text_len
df.head()

,target,text,text_len,words,density
0,0,obama forge his muslim alliance against the ci...,104,22,0.211538
1,4,had the most spectacular prom ever but now my ...,127,25,0.196850
2,0,i am overwhelmed today taking a moment to eat ...,55,12,0.218182
3,0,lindork tres sad i wa totally a max fan sytycd,46,10,0.217391
4,0,crap i wa counting down the hour until my dad ...,132,30,0.227273


In [ ]:
# Feature Construction: Noun count
# ---
# YOUR CODE GOES BELOW
#
# First, we will download the punkt and the averaged_perceptron_tagger into our notebook environment. 
# which will allow us to find the part of speech tags.
# ---
#
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# We create the function to check and get the part of speech tag count of a words in a given sentence
def pos_check(x, flag="noun"):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

In [ ]:
# Noun Count
# ---
# YOUR CODE GOES BELOW
#
df["noun"]=df.text.apply(pos_check)
df.head()

,target,text,text_len,words,density,noun
0,0,obama forge his muslim alliance against the ci...,104,22,0.211538,0
1,4,had the most spectacular prom ever but now my ...,127,25,0.196850,0
2,0,i am overwhelmed today taking a moment to eat ...,55,12,0.218182,0
3,0,lindork tres sad i wa totally a max fan sytycd,46,10,0.217391,0
4,0,crap i wa counting down the hour until my dad ...,132,30,0.227273,0


In [ ]:
# Feature Construction: Verb count
# ---
# YOUR CODE GOES BELOW
#
def pos_check(x, flag="verb"):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt
df["verb"]=df.text.apply(pos_check)

In [ ]:
# Feature Construction: Adjective count / Tweet
# ---
# YOUR CODE GOES BELOW
#
def pos_check(x, flag="adj"):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt
df["adj"]=df.text.apply(pos_check)

In [ ]:
# Feature Construction: Adverb count / Tweet
# ---
# YOUR CODE GOES BELOW
#
def pos_check(x, flag="adv"):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt
df["adv"]=df.text.apply(pos_check)

In [ ]:
# Feature Construction: Pronoun 
# ---
# YOUR CODE GOES BELOW
#
def pos_check(x, flag="pron"):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt
df["pron"]=df.text.apply(pos_check)

In [ ]:
# Feature Construction: Subjectivity
# ---
# YOUR CODE GOES BELOW
# 

# Function to calculate subjectiveity and polarity
def get_subjectivity(text):
    return textblob.TextBlob(text).sentiment.subjectivity

df["subjecttivity"]=df.text.apply(get_subjectivity)
df.head()

,target,text,text_len,words,density,noun,verb,adj,adv,pron,subjecttivity
0,0,obama forge his muslim alliance against the ci...,104,22,0.211538,0,0,0,0,0,0.900000
1,4,had the most spectacular prom ever but now my ...,127,25,0.196850,0,0,0,0,0,0.762500
2,0,i am overwhelmed today taking a moment to eat ...,55,12,0.218182,0,0,0,0,0,0.000000
3,0,lindork tres sad i wa totally a max fan sytycd,46,10,0.217391,0,0,0,0,0,0.875000
4,0,crap i wa counting down the hour until my dad ...,132,30,0.227273,0,0,0,0,0,0.474074


In [ ]:
# Feature Construction: Polarity
# ---
# YOUR CODE GOES BELOW
# 
def get_polarity(text):
    return textblob.TextBlob(text).sentiment.polarity

df["polarity"]=df.text.apply(get_polarity)
df.head()

,target,text,text_len,words,density,noun,verb,adj,adv,pron,subjecttivity,polarity
0,0,obama forge his muslim alliance against the ci...,104,22,0.211538,0,0,0,0,0,0.900000,0.400000
1,4,had the most spectacular prom ever but now my ...,127,25,0.196850,0,0,0,0,0,0.762500,0.612500
2,0,i am overwhelmed today taking a moment to eat ...,55,12,0.218182,0,0,0,0,0,0.000000,0.000000
3,0,lindork tres sad i wa totally a max fan sytycd,46,10,0.217391,0,0,0,0,0,0.875000,-0.250000
4,0,crap i wa counting down the hour until my dad ...,132,30,0.227273,0,0,0,0,0,0.474074,-0.235185


In [ ]:
# Feature Construction: Word Level N-Gram TF-IDF Feature 
# ---
# YOUR CODE GOES BELOW
#
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(1, 1), max_features=2000)

df_char_vect = vec.fit_transform(df.text)

In [ ]:
# Feature Construction: Character Level N-Gram TF-IDF Feature
# ---
# YOUR CODE GOES BELOW
# 
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(ngram_range=(2, 2), max_features=2000)

df_word_vect = vec.fit_transform(df.text)

In [ ]:
# Let's prepare the constructed features for modeling
# ---
#
X_metadata = np.array(df.iloc[:, 2:12])
X_metadata

array([[104.        ,  22.        ,   0.21153846, ...,   0.        ,
          0.9       ,   0.4       ],
       [127.        ,  25.        ,   0.19685039, ...,   0.        ,
          0.7625    ,   0.6125    ],
       [ 55.        ,  12.        ,   0.21818182, ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [ 56.        ,  13.        ,   0.23214286, ...,   0.        ,
          0.56666667,   0.26666667],
       [ 40.        ,   8.        ,   0.2       , ...,   0.        ,
          0.56785714,   0.49285714],
       [ 65.        ,  14.        ,   0.21538462, ...,   0.        ,
          0.        ,   0.        ]])

In [ ]:
# We combine our two tfidf (sparse) matrices and X_metadata
# ---
#
X = scipy.sparse.hstack([df_word_vect, df_char_vect,  X_metadata]).toarray()
X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.9       ,
        0.4       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.7625    ,
        0.6125    ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.56666667,
        0.26666667],
       [0.        , 0.        , 0.        , ..., 0.        , 0.56785714,
        0.49285714],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
# Getting our response variable
# ---
#
y = np.array(df.iloc[:, 0])
y

array([0, 4, 0, ..., 0, 4, 0], dtype=int64)

### 4. Data Modelling

During this step, we will use machine learning algorithms to train and test our sentiment analysis models.

In [ ]:
# Splitting our data
# ---
#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
# Fitting our model
# ---
#

# Importing the algorithms
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

nb_classifier = MultinomialNB() 
lr_classifier = LogisticRegression(max_iter=1000) 

# Training our model
nb_classifier.fit(X_train, y_train) 
lr_classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
# Making predictions
# ---
#
y_predict_nb = nb_classifier.predict(X_test) 
y_predict_lr = lr_classifier.predict(X_test)

In [ ]:
# Evaluating the Models
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Accuracy scores
# ---
#
print("Naive Bayes Classifier:\n", accuracy_score(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", accuracy_score(y_test, y_predict_lr))

Naive Bayes Classifier:
 0.7475
Logistic Regression Classifier: 
 0.749


In [ ]:
# Confusion matrices
# ---
# 
print("Naive Bayes Classifier: \n", confusion_matrix(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", confusion_matrix(y_test, y_predict_lr))

Naive Bayes Classifier: 
 [[782 268]
 [237 713]]
Logistic Regression Classifier: 
 [[829 221]
 [281 669]]


In [ ]:
# Classification Reports
# ---
#
print("Naive Bayes Classifier: \n", classification_report(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", classification_report(y_test, y_predict_lr))

Naive Bayes Classifier: 
               precision    recall  f1-score   support

           0       0.77      0.74      0.76      1050
           4       0.73      0.75      0.74       950

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000

Logistic Regression Classifier: 
               precision    recall  f1-score   support

           0       0.75      0.79      0.77      1050
           4       0.75      0.70      0.73       950

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000



**Evaluation our Models**

* **Accuracy:** the percentage of texts that were assigned the correct topic.
* **Precision:** the percentage of texts the classifier classified correctly out of the total number of texts it predicted for each topic
* **Recall:** the percentage of texts the model predicted for each topic out of the total number of texts it should have predicted for that topic.
* **F1 Score:** the average of both precision and recall.

To improve our model, we can try perfoming other text processing techniques that would better prepare our data for fitting our model. We can also use different vectorizing techniques, implement other machine learning models and perform hyperparameter tuning.

### 5. Recommendations


Our best model had an accuracy of 73.25% and use it for classifying newer tweets. We can improve this performance by performing hyperparameter tuning and feature engineering methods. 